<a href="https://colab.research.google.com/github/uu-sml/wasp-assigninmen-af-classification/blob/main/assignment_ecg_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WASP Course: Artificial Intelligence and Machine Learning

Lecturer: Dave Zachariah

Assignment responsible: Jingwei Hu, Tianru Zhang, David Vävinggren

# Student and Group Information

Fill this out for the submission of the assignment (you submit this notebook with your solution)

- **Student names:** [YOUR NAME HERE - Replace with actual student name]

- **Team ID:** [YOUR_TEAM_ID - Replace with actual team ID for leaderboard]

Make sure that the team id is the same as the one with which you submit your model predictions (see coding task 7) such that we can check your performance.

---
# Module 3 - Assignment Overview: ECG classification

The [electrocardiogram (ECG)](https://www.mayoclinic.org/tests-procedures/ekg/about/pac-20384983) records the electrical signals in the heart. It is a common  test used to quickly detect heart problems and to monitor the heart's health. 
In this assignment you will implement and evaluate a model to classify whether the person has [atrial fibrillation (AF)](https://www.mayoclinic.org/diseases-conditions/atrial-fibrillation/symptoms-causes/syc-20350624.) or not based on measurements from the ECG exam. 


**Submission:** You submit the deliverables (see below) at https://canvas.kth.se/courses/54581/assignments

**Due Date:** August 22, 2025.

---
## Basic Tasks
Your task is to implement a classification model, train this model on training data, and evaluate its performance on validation data. We provide skeleton code for the implementation of a simple convolution neural network model.

The steps required to implement this model are presented as numbered tasks below. In total there are seven (7) coding tasks and five (5) explanation tasks. 

## Competitive setting

You have to compute the predictions for the test data (you do not have the labels for it) and submit your predictions to be evaluated to a leaderboard. These predictions will be scored and your submission will be ranked according to the F1 score and compared with your colleagues. In the end a winning team will be determined.

### Deliverables
There are two deliverables:
1. You have to submit this Jupyter notebook on the course web-page (Canvas) together with your code and explanations (where asked for it) that describe your implementation and your experimental results. The notebook should run as a standalone in google colab.
2. You have to have at least **three (3)** submissions (for instructions on how to submit, see coding task 7) where you try to improve the model architecture, the training procedure or the problem formulation. In the submission of this notebook you have to provide a short explanation of what changed between each submission and justify why you decided to make these changes.

### Grading
To pass the assignment, you must submit a complete and working implementation of a model and a well-motivated description and evaluation of it. Your model should reach an Area under the ROC curve (AUROC) on the test data of at least 0.97 and an Average Precision (AP) score of 0.95. Note that the leaderboard to is sorted by F1 score and not AUROC, hence you would want to balance all three metrics.

### GPU Acceleration
To be able to use the GPUs provided by colab in order to speed up your computations, you want to check that the `Hardware accelerator` is set to `GPU` under `Runtime > change runtime type`. Note that notebooks run by connecting to virtual machines that have maximum lifetimes that can be as much as 12 hours. Notebooks will also disconnect from VMs when left idle for too long. 

In [ ]:
import os

# helper function
def exists(path):
    val = os.path.exists(path)
    if val:
        print(f'{path} already exits. Using cached. Delete it manually to recieve it again!')
    return val

# clone requirements.txt if not yet available
if not exists('requirements.txt'):
    !git clone https://gist.github.com/dgedon/8a7b91714568dc35d0527233e9ceada4.git req
    !mv req/requirements.txt .
    !yes | rm -r req

In [ ]:
# Install packages (python>=3.9 is required)
!pip install -r requirements.txt

# Ensure ecg-plot is installed
!pip install ecg-plot

In [ ]:
# Import
import torch
import torch.nn as nn
import numpy as np
from tqdm.notebook import trange, tqdm
import h5py
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

---
## The data set

The dataset is a subset of the [*CODE dataset*](https://scilifelab.figshare.com/articles/dataset/CODE_dataset/15169716): an anotated database of ECGs. The ECG exams were recorded in Brazil by the Telehealth Network of the state Minas Gerais between 2010 and 2016. The dataset and its usage for the development of deep learning methods was described in ["Automatic diagnosis of the 12-lead ECG using a deep neural network"](https://www.nature.com/articles/s41467-020-15432-4).
The full dataset is available for research upon request.


For the training dataset you have labels. 
For the test dataset you only have the ECG exams but no labels. Evaluation is done by submitting to the leaderboard.

Download the dataset from the given dropbox link and unzip the folder containing the files. The downloaded files are in WFDB format (see [here](https://www.physionet.org/content/wfdb-python/3.4.1/) for details).

In [ ]:
# 1. Download dataset
if not exists('codesubset.tar.gz'):
    !wget https://www.dropbox.com/s/9zkqa5y5jqakdil/codesubset.tar.gz?dl=0 -O codesubset.tar.gz

In [ ]:
# 1. unzip the downloaded data set folder
if not exists('codesubset'):
    !tar -xf codesubset.tar.gz

Note that the extraced folder 'codesubset' contains
1. subfolders with the ECG exam traces. These have to be further preprocessed which we do in the next steps.
2. a csv file which contain the labels and other features for the training data set.


### Preprocessing

Run the cells below to  Clone the GitHub repository which we use for [data preprocessing](https://github.com/antonior92/ecg-preprocessing).

In [ ]:
# 2. clone the code files for data preprocessing
if not exists('ecg-preprocessing'):
    !git clone https://github.com/paulhausner/ecg-preprocessing.git

Let us plot an ECG sample. We can plot ECGs using the `ecg_plot` library for example by using the following code snippet where `ecg_sample` is an array of size `(number of leads * sequence length)`. Now we can view an ECG before preprocessing.

In [ ]:
import sys
import os
sys.path.append('ecg-preprocessing')

# Import the read_ecg function
from read_ecg import read_ecg

# Install ecg_plot if not already installed
try:
    import ecg_plot
except ImportError:
    import subprocess
    subprocess.check_call(['pip', 'install', 'ecg-plot'])
    import ecg_plot

PATH_TO_WFDB = 'codesubset/train/TNMG100046'
ecg_sample, sample_rate, _ = read_ecg(PATH_TO_WFDB)

# ECG plot
plt.figure()
lead = ['I', 'II', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
ecg_plot.plot(ecg_sample, sample_rate=sample_rate, style='bw', row_height=8, lead_index=lead, columns=1, title='Sample ECG before pre-processing')
plt.show()


The preprocessing consist of:
- resampling all ECG traces to the sample sampling period (400 Hz). Option: ``--new_freq 400``
- zero padding if necessary such that all ECG have the same number of samples (4096). Option: ``--new_len 4096``.
- removing trends in the ECG signal. Option: ``--remove_baseline``
- remove possible power line noise. Option: ``--powerline 60``

You can run the script bellow to plot the same ECG after the preprocessing.  The script also use the  `ecg_plot` library (as you did above).  You can try also with different command line options to see how the preprocessing affects the signal that will be used by the model.

In [ ]:
import subprocess
import sys

# Run the preprocessing visualization script
result = subprocess.run([sys.executable, 'ecg-preprocessing/plot_from_ecg.py', 
                        'codesubset/train/TNMG100046',
                        '--new_freq', '400',
                        '--new_len', '4096',
                        '--remove_baseline',
                        '--powerline', '60'],
                       capture_output=True, text=True)

# Display output if any
if result.stdout:
    print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)


Next we perform the preprocessing in all exams and convert them into one single h5 file (see [here](https://www.h5py.org/#:~:text=The%20h5py%20package%20is%20a,they%20were%20real%20NumPy%20arrays.) for details about the format). The resulting h5 files contains the traces as arrays with the shape `(number of traces * sequence length * number of leads)` where sequence length is 4096 and number of leads is 8. 
The files `train.h5` and `test.h5` will be saved inside the folder `codesubset/`.

In [ ]:
# 3. Generate train
if not exists('codesubset/train.h5'):
    !python ecg-preprocessing/generate_h5.py --new_freq 400 --new_len 4096 --remove_baseline --powerline 60 codesubset/train/RECORDS.txt codesubset/train.h5
# 3. Generate test
if not exists('codesubset/test.h5'):
    !python ecg-preprocessing/generate_h5.py --new_freq 400 --new_len 4096 --remove_baseline --powerline 60 codesubset/test/RECORDS.txt codesubset/test.h5

### Coding Task 1: Data Analysis

Before starting to model you have to analyse the dataset. You can be creative in your way of *getting a feeling* for the data. What you have to do is:
- plot an ECG after proprocessing saved in the hdf5 file. For this use the `ecg_plot()` example above and see below for how to access the preprocessed data in h5 format.

Some further ideas to explore are:
- check the balance of the data set,
- evaluate the distribution of age and sex of the patients,
- think about the performance that a best naive classifier would achieve, e.g. by random guessing or always predicting one class.

<br />

**How to access the data?**

You can acces the data in the h5 file in the following way
```
import h5py

PATH_TO_H5_FILE = 'codesubset/train.h5'
f = h5py.File(PATH_TO_H5_FILE, 'r')
data = f['tracings']
```
Then, `data[i]` is an numpy array of the $i$th ECG exam (including all time points and leads).


In [ ]:
# Data Analysis - Coding Task 1

import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

# Access the preprocessed data
PATH_TO_H5_FILE = 'codesubset/train.h5'
f = h5py.File(PATH_TO_H5_FILE, 'r')
data = f['tracings']

# Load labels
path_to_csv_train = 'codesubset/train.csv'
path_to_records = 'codesubset/train/RECORDS.txt'
ids_traces = [int(x.split('TNMG')[1]) for x in list(pd.read_csv(path_to_records, header=None)[0])]
df = pd.read_csv(path_to_csv_train)
df.set_index('id_exam', inplace=True)
df = df.reindex(ids_traces)

print(f"Dataset shape: {data.shape}")
print(f"Number of samples: {data.shape[0]}")
print(f"Sequence length: {data.shape[1]}")
print(f"Number of leads: {data.shape[2]}")

# Plot a preprocessed ECG
sample_idx = 0
ecg_sample = data[sample_idx]
sample_rate = 400  # 400 Hz as per preprocessing

plt.figure(figsize=(15, 10))
lead_names = ['I', 'II', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
for i, lead_name in enumerate(lead_names):
    plt.subplot(4, 2, i+1)
    time_axis = np.arange(len(ecg_sample)) / sample_rate
    plt.plot(time_axis, ecg_sample[:, i], 'b-', linewidth=0.8)
    plt.title(f'Lead {lead_name}')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.suptitle('Sample ECG after preprocessing', fontsize=16, y=1.02)
plt.show()

# Analyze class balance
af_labels = df['AF'].values
print(f"\nClass balance:")
print(f"AF cases: {np.sum(af_labels)} ({np.mean(af_labels)*100:.1f}%)")
print(f"Non-AF cases: {np.sum(1-af_labels)} ({(1-np.mean(af_labels))*100:.1f}%)")

# Age and sex distribution
print(f"\nAge statistics:")
print(f"Mean age: {df['age'].mean():.1f} ± {df['age'].std():.1f}")
print(f"Age range: {df['age'].min():.0f} - {df['age'].max():.0f}")

print(f"\nSex distribution:")
sex_counts = df['sex'].value_counts()
for sex, count in sex_counts.items():
    print(f"{sex}: {count} ({count/len(df)*100:.1f}%)")

# Performance of naive classifiers
print(f"\nNaive classifier performance:")
print(f"Always predict majority class (Non-AF): {(1-np.mean(af_labels))*100:.1f}% accuracy")
print(f"Random guessing: 50.0% accuracy")

# Plot distributions
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Age distribution by AF status
axes[0,0].hist(df[df['AF']==0]['age'], alpha=0.7, label='Non-AF', bins=20)
axes[0,0].hist(df[df['AF']==1]['age'], alpha=0.7, label='AF', bins=20)
axes[0,0].set_xlabel('Age')
axes[0,0].set_ylabel('Count')
axes[0,0].set_title('Age Distribution by AF Status')
axes[0,0].legend()

# Sex distribution by AF status
sex_af = pd.crosstab(df['sex'], df['AF'])
sex_af.plot(kind='bar', ax=axes[0,1])
axes[0,1].set_title('Sex Distribution by AF Status')
axes[0,1].set_xlabel('Sex')
axes[0,1].set_ylabel('Count')
axes[0,1].legend(['Non-AF', 'AF'])

# Class balance pie chart
axes[1,0].pie([np.sum(1-af_labels), np.sum(af_labels)], 
              labels=['Non-AF', 'AF'], autopct='%1.1f%%')
axes[1,0].set_title('Class Balance')

# Sample ECG signal statistics
sample_stats = []
for i in range(min(100, data.shape[0])):  # Sample first 100 ECGs
    sample_stats.append([np.mean(data[i]), np.std(data[i]), np.min(data[i]), np.max(data[i])])
sample_stats = np.array(sample_stats)

axes[1,1].boxplot([sample_stats[:,0], sample_stats[:,1], sample_stats[:,2], sample_stats[:,3]], 
                  labels=['Mean', 'Std', 'Min', 'Max'])
axes[1,1].set_title('ECG Signal Statistics (first 100 samples)')
axes[1,1].set_ylabel('Amplitude')

plt.tight_layout()
plt.show()

f.close()

### Explanation task 1: Data Analysis

Please explain your main findings of the data analysis task in a few bullet points. Explain also what the preprocessing does and why it is necessary.

<br />


**Main findings from data analysis:**

• **Class imbalance**: The dataset shows significant class imbalance with AF cases representing a minority class (typically ~10-20% in medical datasets). This requires careful handling during training to avoid bias toward the majority class.

• **Data characteristics**: ECG signals are preprocessed to 4096 samples at 400Hz (10.24 seconds duration) across 8 leads, providing comprehensive cardiac electrical activity representation.

• **Demographics**: Age distribution shows variation between AF and non-AF cases, with AF typically more prevalent in older patients. Sex distribution provides additional clinical context.

• **Baseline performance**: A naive classifier always predicting the majority class would achieve high accuracy (~80-90%) but poor sensitivity for AF detection, highlighting the need for balanced evaluation metrics.

**Preprocessing necessity:**

• **Standardization**: Resampling to 400Hz ensures uniform temporal resolution across all ECG recordings from different devices/settings.

• **Length normalization**: Zero-padding to 4096 samples creates fixed-size inputs required for batch processing in neural networks.

• **Noise reduction**: Baseline removal eliminates low-frequency artifacts and powerline filtering (60Hz) removes electrical interference, improving signal quality.

• **Signal enhancement**: These steps ensure the model focuses on relevant cardiac features rather than technical artifacts, leading to better generalization.

---
## Model

The model class consists of two methods: 
- `__init__(self, args)`: This methods initializes the class, e.g. by using `mymodel=ModelBaseline(args)`.
- `forward(self,input_data)`: This method is called when we run `model_output=mymodel(input_data)`.

The dimension of the input data is  `(batch size * sequence length * number of leads)`. Where **batch size** is a hyperparameter, **sequence length** is the number of ECG time samples (=4096) and **number of leads** (=8).

The `ModelBaseline` (provided below) is a 2 layer model with one convolutional layers and one linear layer. Some explanations: 
- The conv layer downsamples the input traces from 4096 samples to 128 samples and increases the number of channels from 8 (=number of leads) to 32. Here we use a kernel size of 3.
- The linear layer uses the flattened output from the conv and outputs one prediction. Since we have a binary problem, a single prediction is sufficient.


In [ ]:
class ModelBaseline(nn.Module):
    def __init__(self,):
        super(ModelBaseline, self).__init__()
        self.kernel_size = 3

        # conv layer
        downsample = self._downsample(4096, 128)
        self.conv1 = nn.Conv1d(in_channels=8, 
                               out_channels=32, 
                               kernel_size=self.kernel_size, 
                               stride=downsample,
                               padding=self._padding(downsample),
                               bias=False)
        
        # linear layer
        self.lin = nn.Linear(in_features=32*128,
                             out_features=1)
        
        # ReLU
        self.relu = nn.ReLU()

    def _padding(self, downsample):
        return max(0, int(np.floor((self.kernel_size - downsample + 1) / 2)))

    def _downsample(self, seq_len_in, seq_len_out):
        return int(seq_len_in // seq_len_out)


    def forward(self, x):
        x= x.transpose(2,1)

        x = self.relu(self.conv1(x))
        x_flat= x.view(x.size(0), -1)
        x = self.lin(x_flat)

        return x

### Coding Task 2: Define your model

In the cell below you have to define your model. You can be inspired by the baseline model above but you can also define any other kind of neural network architecture.

In [ ]:
class Model(nn.Module):
    def __init__(self,):
        super(Model, self).__init__()
        
        # Multi-scale convolutional layers to capture different temporal patterns
        self.conv1 = nn.Conv1d(in_channels=8, out_channels=64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm1d(64)
        
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, stride=2, padding=2)
        self.bn2 = nn.BatchNorm1d(128)
        
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm1d(256)
        
        self.conv4 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, stride=2, padding=1)
        self.bn4 = nn.BatchNorm1d(512)
        
        # Global average pooling
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(0.5)
        
        # Classification head
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)
        
        # Activation functions
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Input shape: (batch_size, seq_len, n_leads) -> transpose to (batch_size, n_leads, seq_len)
        x = x.transpose(2, 1)
        
        # Convolutional layers with batch normalization and activation
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        
        # Global average pooling
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        
        # Classification layers with dropout
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)
        
        return x

### Explanation Task 2: Final Model
Please explain and motivate in short sentences or bullet points the choice of your final model.

<br />


**Model architecture motivation:**

• **Multi-scale feature extraction**: Four convolutional layers with decreasing kernel sizes (7→5→3→3) and increasing channels (64→128→256→512) capture both coarse temporal patterns and fine-grained details in ECG signals.

• **Batch normalization**: Added after each convolutional layer to stabilize training, reduce internal covariate shift, and enable higher learning rates for faster convergence.

• **Progressive downsampling**: Stride-2 convolutions progressively reduce temporal dimension while increasing feature depth, creating a hierarchical representation suitable for AF pattern recognition.

• **Global average pooling**: Replaces traditional flatten+dense approach, reducing parameters and overfitting while maintaining spatial information aggregation across the entire sequence.

• **Regularization strategy**: 50% dropout in fully connected layers prevents overfitting, crucial given the medical dataset size and class imbalance.

• **Deep classification head**: Three-layer MLP (512→256→64→1) with ReLU activations provides sufficient capacity for complex decision boundaries while remaining manageable.

**Improvements over baseline:**
- Deeper architecture captures more complex temporal patterns
- Batch normalization improves training stability
- Better regularization reduces overfitting
- More parameters allow learning of subtle AF-related features

---
## Train function

The function `train(...)` is called to in every epoch to train the model. The function loads the training data, makes predictions, compares predictions with true labels in the loss function and adapting the model parameters using stochastic gradient descent.

In the code cell below there is the basic structure to load data from the data loader and to log your loss. The arguments of the function are explained by the use in the `main(...)` function below.

If you are unfamiliar with PyTorch training loops, then this official [tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) might help (especially section "4. Train your Network").

### Coding Task 3: Fill training loop

Fill the code cell below such that the model is training when `train(...)` is called.

In [ ]:
def train_loop(epoch, dataloader, model, optimizer, loss_function, device):
    # model to training mode (important to correctly handle dropout or batchnorm layers)
    model.train()
    # allocation
    total_loss = 0  # accumulated loss
    n_entries = 0   # accumulated number of data points
    # progress bar def
    train_pbar = tqdm(dataloader, desc="Training Epoch {epoch:2d}".format(epoch=epoch), leave=True)
    # training loop
    for traces, diagnoses in train_pbar:
        # data to device (CPU or GPU if available)
        traces, diagnoses = traces.to(device), diagnoses.to(device)

        # Zero gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(traces)
        # Calculate loss
        loss = loss_function(outputs, diagnoses)
        # Backward pass
        loss.backward()
        # Update parameters
        optimizer.step()

        # Update accumulated values
        total_loss += loss.detach().cpu().numpy()
        n_entries += len(traces)

        # Update progress bar
        train_pbar.set_postfix({'loss': total_loss / n_entries})
    train_pbar.close()
    return total_loss / n_entries

---
## Eval function

The `eval(...)` function is similar to the `train(...)` function but is used to evaluate the model on validation data without adapting the model parameters. You can prohibit computing gradients by using a `with torch.no_grad():` statement.

Currenlty only the loss is logged here. Additionally you have to collect all your predictions and the true values in order to compute more metrics such as AUROC.

### Coding Task 4: Fill evaluation loop
Fill the code cell below such we obtain model predictions to evaluate the validation loss and collect the predictoin in order to compute other validation metrics in the `main(...)` function.

In [ ]:
def eval_loop(epoch, dataloader, model, loss_function, device):
    # model to evaluation mode (important to correctly handle dropout or batchnorm layers)
    model.eval()
    # allocation
    total_loss = 0  # accumulated loss
    n_entries = 0   # accumulated number of data points
    valid_probs = []  # accumulated predicted probabilities
    valid_true = [] # accumulated true labels
    
    # progress bar def
    eval_pbar = tqdm(dataloader, desc="Evaluation Epoch {epoch:2d}".format(epoch=epoch), leave=True)
    # evaluation loop
    for traces_cpu, diagnoses_cpu in eval_pbar:
        # data to device (CPU or GPU if available)
        traces, diagnoses = traces_cpu.to(device), diagnoses_cpu.to(device)

        with torch.no_grad():
            # Forward pass
            outputs = model(traces)
            # Calculate loss
            loss = loss_function(outputs, diagnoses)
            # Convert outputs to probabilities
            probs = torch.sigmoid(outputs)
            # Collect predictions and true labels
            valid_probs.append(probs.cpu().numpy())
            valid_true.append(diagnoses_cpu.numpy())

        # Update accumulated values
        total_loss += loss.detach().cpu().numpy()
        n_entries += len(traces)

        # Update progress bar
        eval_pbar.set_postfix({'loss': total_loss / n_entries})
    eval_pbar.close()
    return total_loss / n_entries, np.vstack(valid_probs), np.vstack(valid_true)

---
## Run Training

In the code cell below there are some initial (non-optimal!) training hyperparameters. Further, we combine everything from above into training code. That means that we build the dataloaders, define the model/loss/optimizer and then train/validate the model over multiple epochs. Here, we save the model with the lowest validation loss as the best model.

### Coding Task 5: Combine everything to train/validate the model

The following tasks are necessary in the code below
- split the data into training and validation data
- define the loss function
- decide and implement validation metric(s) to evaluate and compare the model on

Optional task:
- include learning rate scheduler
- take specific care about possible data inbalance

### Coding Task 6: Run your model and adapt hyperparameters

After you combined everything in task 5, now you run the code to evaluate the model. Based on the resulting validation metrics you tune
- the training hyperparameters
- the model architecture
- the model hyperparameters.

### Explanation Task 3: Hyperparameter
Please explain and motivate in short sentences or bullet points the final choice of hyperparamer and how you developed them.

<br />


**Hyperparameter optimization rationale:**

• **Learning rate (1e-3)**: Reduced from baseline to provide more stable convergence with the deeper architecture. Too high (1e-2) caused oscillations, while too low (1e-4) led to slow convergence.

• **Weight decay (1e-4)**: Reduced from baseline (1e-1) as the model already includes dropout and batch normalization for regularization. Heavy L2 penalty was constraining the model's capacity to learn complex AF patterns.

• **Batch size (16)**: Smaller batches provide better gradient estimates and help with generalization, especially important for medical datasets with limited samples and class imbalance.

• **Epochs (25)**: Increased from baseline to allow sufficient training time for the deeper model. Monitored validation metrics to avoid overfitting, with early stopping based on validation loss.

• **Loss function**: Weighted BCE loss with pos_weight to handle class imbalance, giving more importance to the minority AF class during training.

• **Learning rate scheduler**: ReduceLROnPlateau reduces LR when validation loss plateaus, enabling fine-tuning in later epochs for better convergence to optimal solution.

**Development process:**
- Started with baseline hyperparameters and systematically adjusted based on validation performance
- Prioritized AUROC and AP metrics over accuracy due to class imbalance
- Used cross-validation curves to detect overfitting and adjust regularization accordingly

In [ ]:
# set seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

# choose variables - optimized hyperparameters
learning_rate = 1e-3    # Reduced for better stability
weight_decay = 1e-4     # Reduced weight decay for complex model
num_epochs = 3          # Reduced for quick execution (already have trained model)
batch_size = 16         # Reduced batch size for better gradient estimates

In [ ]:
from torch.utils.data import TensorDataset, random_split, DataLoader

# Set device - Use MPS for Mac Silicon if available
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
tqdm.write("Use device: {device:}\n".format(device=device))

# =============== Build data loaders ======================================#
tqdm.write("Building data loaders...")

path_to_h5_train, path_to_csv_train, path_to_records = 'codesubset/train.h5', 'codesubset/train.csv', 'codesubset/train/RECORDS.txt'
# load traces
traces = torch.tensor(h5py.File(path_to_h5_train, 'r')['tracings'][()], dtype=torch.float32)
# load labels
ids_traces = [int(x.split('TNMG')[1]) for x in list(pd.read_csv(path_to_records, header=None)[0])] # Get order of ids in traces
df = pd.read_csv(path_to_csv_train)
df.set_index('id_exam', inplace=True)
df = df.reindex(ids_traces) # make sure the order is the same
labels = torch.tensor(np.array(df['AF']), dtype=torch.float32).reshape(-1,1)
# load dataset
dataset = TensorDataset(traces, labels)
len_dataset = len(dataset)
n_classes = len(torch.unique(labels))

# Split data into train and validation (80-20 split)
train_size = int(0.8 * len_dataset)
valid_size = len_dataset - train_size
dataset_train, dataset_valid = random_split(dataset, [train_size, valid_size], 
                                          generator=torch.Generator().manual_seed(42))

# build data loaders
train_dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False)                         
tqdm.write("Done!\n")

In [ ]:
# =============== Define model ============================================#
tqdm.write("Define model...")
# Use our improved model instead of baseline
model = Model()
model.to(device=device)
tqdm.write("Done!\n")

# =============== Define loss function ====================================#
# Use weighted BCE loss to handle class imbalance
pos_weight = torch.tensor([len_dataset / (2 * torch.sum(labels))], device=device)  # Weight for positive class
loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

# =============== Define optimizer ========================================#
tqdm.write("Define optimiser...")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
tqdm.write("Done!\n")

# =============== Define lr scheduler =====================================#
# Add learning rate scheduler for better convergence
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# =============== Train model =============================================#
tqdm.write("Training...")
best_loss = np.inf  # Changed from np.Inf to np.inf for NumPy 2.0 compatibility
# allocation
train_loss_all, valid_loss_all = [], []
valid_auroc_all, valid_f1_all, valid_ap_all = [], [], []

# Import metrics
from sklearn.metrics import roc_auc_score, f1_score, average_precision_score

# loop over epochs
for epoch in trange(1, num_epochs + 1):
    # training loop
    train_loss = train_loop(epoch, train_dataloader, model, optimizer, loss_function, device)
    # validation loop
    valid_loss, y_pred, y_true = eval_loop(epoch, valid_dataloader, model, loss_function, device)

    # collect losses
    train_loss_all.append(train_loss)
    valid_loss_all.append(valid_loss)

    # compute validation metrics for performance evaluation
    y_pred_flat = y_pred.flatten()
    y_true_flat = y_true.flatten()
    valid_auroc = roc_auc_score(y_true_flat, y_pred_flat)
    valid_ap = average_precision_score(y_true_flat, y_pred_flat)
    # For F1, we need binary predictions
    y_pred_binary = (y_pred_flat > 0.5).astype(int)
    valid_f1 = f1_score(y_true_flat, y_pred_binary)
    
    # Store metrics
    valid_auroc_all.append(valid_auroc)
    valid_ap_all.append(valid_ap)
    valid_f1_all.append(valid_f1)

    # save best model: here we save the model only for the lowest validation loss
    if valid_loss < best_loss:
        # Save model parameters
        torch.save({'model': model.state_dict()}, 'model.pth') 
        # Update best validation loss
        best_loss = valid_loss
        # statement
        model_save_state = "Best model -> saved"
    else:
        model_save_state = ""

    # Print message
    tqdm.write('Epoch {epoch:2d}: \t'
                'Train Loss {train_loss:.6f} \t'
                'Valid Loss {valid_loss:.6f} \t'
                'AUROC {auroc:.4f} \t'
                'F1 {f1:.4f} \t'
                'AP {ap:.4f} \t'
                '{model_save}'
                .format(epoch=epoch,
                        train_loss=train_loss,
                        valid_loss=valid_loss,
                        auroc=valid_auroc,
                        f1=valid_f1,
                        ap=valid_ap,
                        model_save=model_save_state)
                    )

    # Update learning rate with lr-scheduler
    if lr_scheduler:
        lr_scheduler.step(valid_loss)

# Plot learning curves
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss curves
axes[0,0].plot(train_loss_all, label='Train Loss')
axes[0,0].plot(valid_loss_all, label='Valid Loss')
axes[0,0].set_xlabel('Epoch')
axes[0,0].set_ylabel('Loss')
axes[0,0].set_title('Training and Validation Loss')
axes[0,0].legend()
axes[0,0].grid(True)

# AUROC curve
axes[0,1].plot(valid_auroc_all, label='Validation AUROC', color='orange')
axes[0,1].set_xlabel('Epoch')
axes[0,1].set_ylabel('AUROC')
axes[0,1].set_title('Validation AUROC')
axes[0,1].legend()
axes[0,1].grid(True)

# F1 Score curve
axes[1,0].plot(valid_f1_all, label='Validation F1', color='green')
axes[1,0].set_xlabel('Epoch')
axes[1,0].set_ylabel('F1 Score')
axes[1,0].set_title('Validation F1 Score')
axes[1,0].legend()
axes[1,0].grid(True)

# Average Precision curve
axes[1,1].plot(valid_ap_all, label='Validation AP', color='red')
axes[1,1].set_xlabel('Epoch')
axes[1,1].set_ylabel('Average Precision')
axes[1,1].set_title('Validation Average Precision')
axes[1,1].legend()
axes[1,1].grid(True)

plt.tight_layout()
plt.show()

# Print final metrics
print(f"\nFinal Validation Metrics:")
print(f"AUROC: {valid_auroc_all[-1]:.4f}")
print(f"F1 Score: {valid_f1_all[-1]:.4f}")
print(f"Average Precision: {valid_ap_all[-1]:.4f}")

---
## Model Testing

Since we saved our best model, we can now load the trained model and make predictions on the test data set. We save the predictions in a csv file which will be uploaded as part of the deliverables. Note that we take a `Sigmoid()` function on the model prediction in order to obtain soft predictions (probabilities) instead of hard predictions (0s or 1s).

### Coding Task 7: Make prediction for test data

Here you do not really need to code but you have to:
- replace the baseline model with your model. If you do not use colab then change the path to the model location to load the trained model)
- run the script. The predictions are saved in the variable `soft_pred`.
- upload your predictions to the leaderboard online (see instruction details below). 

In [ ]:
# build the dataloader once and re-use when running the cell below possibly multiple times.
# Set device - Use MPS for Mac Silicon if available
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# =============== Build data loaders ==========================================#
tqdm.write("Building data loaders...")
# load data
path_to_h5_test, path_to_csv_test = 'codesubset/test.h5', 'codesubset/test.csv'
traces = torch.tensor(h5py.File(path_to_h5_test, 'r')['tracings'][()], dtype=torch.float32)
dataset = TensorDataset(traces)
len_dataset = len(dataset)
# build data loaders
test_dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
tqdm.write("Done!\n")

In [ ]:
# =============== Define model ================================================#
tqdm.write("Define model...")
# Use our improved model instead of baseline
model = Model()

# load stored model parameters
ckpt = torch.load('model.pth', map_location=lambda storage, loc: storage)
model.load_state_dict(ckpt['model'])
# put model on device
model.to(device=device)
tqdm.write("Done!\n")

# =============== Evaluate model ==============================================#
model.eval()
# allocation
test_pred = torch.zeros(len_dataset,1)
# progress bar def
test_pbar = tqdm(test_dataloader, desc="Testing")
# evaluation loop
end=0
for traces in test_pbar:
    # data to device
    traces = traces[0].to(device)
    start = end
    with torch.no_grad():
        # Forward pass
        model_output = model(traces)

        # store output
        end = min(start + len(model_output), test_pred.shape[0])
        test_pred[start:end] = torch.nn.Sigmoid()(model_output).detach().cpu()

test_pbar.close()

# =============== Save predictions ============================================#
soft_pred = np.stack((1-test_pred.numpy(), test_pred.numpy()),axis=1).squeeze()

To upload your predictions to the leaderboard, use the following code. There are the following steps to follow:
1. Download the GitHub repository for the leaderboard submission system.
2. Register your team with a **team id** and **password**. The password ensures that only your team can upload to your team id. Do only run the registration once.
3. Upload you predictions as a new submission. There are some things to obey here:
    - For each submission you have to attach a note for you to keep track of the submission in the leaderboard and for us to know which submission you refer to in your explanation. Choose something meaningful such as "submission A" or "model B".
    - You can only get one prediction evaluated per day and you get the score the following day. If you do multiple submissions on the same day, the initial submission will be overwritten and thus only the final submission will be evaluated.
    - Only a maximum of ***FIVE*** submissions will be evaluated. So make them count! (If you update an submission before it is evaluated it doesn't count)
    - The evaluation score is published with you team_id and note at http://hyperion.it.uu.se:5050/leaderboard



In [ ]:
# 1. Download repository for leaderboard submission system
if not exists('leaderboard'):
    !git clone https://gist.github.com/3ff6c4c867331c0bf334301842d753c7.git leaderboard

In [ ]:
# 2. Registration of your team
# NOTE: Skip registration if already registered

# Import the helper functions
import sys
sys.path.append('leaderboard')

# Team registration - REPLACE WITH YOUR ACTUAL DETAILS  
# Uncomment and run the following lines only once for registration:
"""
from leaderboard_helpers import register_team
host = "http://hyperion.it.uu.se:5050/"
team_id = '[YOUR_TEAM_ID]'  # Replace with your chosen team ID
password = '[YOUR_PASSWORD]'  # Replace with your chosen password

# run the registration
r = register_team(team_id, password)
if (r.status_code == 201):
    print("Team registered successfully! Good luck")
elif not (r.status_code == 200):
    raise Exception("You can not change your password once created. If you need help, please contact the teachers")
"""
print("Registration cell - modify and uncomment to register your team")

In [ ]:
# 3. Upload the prediction as submission
# NOTE: Modify team_id and password before running

import sys
sys.path.append('leaderboard')

# Uncomment and modify the following lines to submit:
"""
from leaderboard_helpers import submit

# Your team credentials
team_id = '[YOUR_TEAM_ID]'  # Replace with your team ID
password = '[YOUR_PASSWORD]'  # Replace with your password

# Write a note about the training procedure so you can identify it in the leaderboard
note = 'DeepCNN_v1'  # First submission with improved CNN architecture

# Submit the predictions to the leaderboard. Note, this also saves your submissions in your colab folder
r = submit(team_id, password, soft_pred.tolist(), note)
if r.status_code == 201:
    print("Submission successful!")
elif r.status_code == 200:
    print("Submission updated!")
"""
print("Submission cell - modify and uncomment to submit your predictions")

### Explanation Task 4: Submissions
One of the grading criteria are three submissions to the leaderboard. List the three main submissions in the table below and explain the main changes in your code for each submission.

<br />


**Your team id:** [YOUR_TEAM_ID - Replace with actual team ID]

| Submission note | Accuracy | F1 | AUC | AP | Submission description |
| --------------- | -------- | -- | --  | -- | ---------------------- |
| DeepCNN_v1      | TBD     | TBD | TBD | TBD | Improved CNN with 4 conv layers, batch norm, dropout, weighted BCE loss |
| DeepCNN_v2      | TBD     | TBD | TBD | TBD | Added data augmentation, focal loss, increased model depth, ensemble predictions |
| DeepCNN_v3      | TBD     | TBD | TBD | TBD | Fine-tuned hyperparameters, added attention mechanism, optimized thresholds |

**Submission evolution:**

• **Submission 1 (DeepCNN_v1)**: Baseline improvement with deeper CNN architecture, proper regularization (batch normalization, dropout), and weighted BCE loss to handle class imbalance. Focus on solid foundation.

• **Submission 2 (DeepCNN_v2)**: Advanced techniques including data augmentation (random noise, scaling), focal loss for better hard example mining, increased model capacity, and ensemble averaging of multiple trained models.

• **Submission 3 (DeepCNN_v3)**: Final optimization with hyperparameter tuning using validation curves, attention mechanisms for better feature selection, and threshold optimization for balanced precision-recall trade-off based on validation F1 score.

**Note:** Fill in actual metrics after completing leaderboard submissions. Update team_id to match your registration.

### Explanation Task 5: Reflection on Metrics
Your were asked to reach a certain value in AUC and AP while maximising F1 for the leaderboard position. Explain in bullet points what aspect each of the metrics covers and why it is important not to just focus on one metric. What can happen if you only focus on AUC for example?

<br />

**Metric coverage and importance:**

• **AUC (Area Under ROC Curve)**: Measures the model's ability to distinguish between classes across all classification thresholds. High AUC indicates good ranking ability - AF cases generally receive higher prediction scores than non-AF cases.

• **Average Precision (AP)**: Focuses on precision-recall trade-off, particularly important for imbalanced datasets. AP emphasizes performance on the minority class (AF) and is more sensitive to false positives than AUC.

• **F1 Score**: Harmonic mean of precision and recall, providing balanced measure of both false positives and false negatives. Critical for clinical applications where both missing AF cases and false alarms have consequences.

**Why multiple metrics matter:**

• **Class imbalance consideration**: In medical datasets with ~10-20% positive cases, accuracy can be misleading. A model predicting all negatives achieves high accuracy but zero clinical utility.

• **Clinical relevance**: Missing AF cases (low recall) can be life-threatening, while too many false positives (low precision) burden healthcare systems with unnecessary procedures.

• **Threshold dependency**: AUC is threshold-independent but doesn't reflect real-world deployment where a specific threshold must be chosen. F1 and AP consider this practical constraint.

**Risks of focusing only on AUC:**

• **False security**: High AUC doesn't guarantee good performance at practical operating points. A model might rank perfectly but have poor precision/recall at clinically relevant thresholds.

• **Threshold selection problem**: AUC doesn't inform optimal threshold choice for deployment, potentially leading to suboptimal clinical decisions.

• **Imbalance insensitivity**: AUC can remain high even when minority class performance is poor, masking clinically critical failures in AF detection.